<a href="https://colab.research.google.com/github/lavithra-mysore/Q-AResume/blob/main/chatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, concatenate_datasets
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# load the dataset
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/UpdatedResumeDataSet.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',                   # Output directory
    num_train_epochs=3,                       # Number of training epochs
    per_device_train_batch_size=2,            # Batch size per device during training
    per_device_eval_batch_size=4,             # Batch size for evaluation
    warmup_steps=500,                         # Number of warmup steps
    weight_decay=0.01,                        # Strength of weight decay
    logging_dir='./logs',                     # Directory for storing logs
    logging_steps=10,                         # Log every N steps
)

# Initialize tokenizer and model for Longformer specifically suited for question answering
model_name = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Initialize a list to store Dataset objects
datasets = []

# Define a function to find the answer positions
def find_answer_positions(context, answer):
    start_index = context.find(answer)
    end_index = start_index + len(answer)
    return start_index, end_index

# Loop through unique categories
for category in df["Category"].unique():
    category_resumes = df[df["Category"] == category]["Resume"].tolist()

    if not category_resumes:
        continue

    # Example question and answer for the category
    question = "What skills are mentioned in the resume?"
    answer = "machine learning"  # Example answer, adjust based on actual data or further extraction logic

    # Prepare context as a simple combination of the resumes
    context = " ".join(category_resumes[:5])  # Limit to first 5 resumes to avoid size issues

    # Tokenize the question and context together with separators
    inputs = tokenizer(question, context, max_length=4096, truncation=True, padding="max_length", return_tensors="pt")

    # Set global attention mask: focus on the question tokens
    global_attention_mask = torch.zeros_like(inputs.attention_mask)
    sep_index = (inputs.input_ids == tokenizer.sep_token_id).nonzero(as_tuple=True)[1][0]
    global_attention_mask[:, :sep_index + 1] = 1

    # Find the start and end positions of the answer in the context
    start_pos, end_pos = find_answer_positions(context, answer)
    # Ensure the positions are tensors matching the batch dimension of input tensors
    start_positions = torch.tensor([start_pos] * inputs.input_ids.size(0))  # Repeat start position for batch
    end_positions = torch.tensor([end_pos] * inputs.input_ids.size(0))      # Repeat end position for batch

    # Create a dictionary containing the tokenized inputs and the labels
    data = {
        "input_ids": inputs.input_ids.squeeze(),
        "attention_mask": inputs.attention_mask.squeeze(),
        "global_attention_mask": global_attention_mask.squeeze(),
        "start_positions": start_positions,
        "end_positions": end_positions
    }

    # Convert the dictionary to a Dataset object
    dataset = Dataset.from_dict(data)

    # Append the Dataset object to the list of datasets
    datasets.append(dataset)

# Concatenate all Dataset objects into a single Dataset
train_dataset = concatenate_datasets(datasets)

# Check if the train_dataset contains samples
if len(train_dataset) == 0:
    raise ValueError("No samples found in the training dataset.")

# Initialize the Trainer with the training dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Try training the model
try:
    trainer.train()
except Exception as e:
    print("Error during training:", e)

# Evaluate the model on the validation set
eval_metrics = trainer.evaluate()

# Print evaluation metrics
print("Evaluation Metrics:")
print("Validation Loss:", eval_metrics["eval_loss"])
print("Accuracy:", eval_metrics["eval_accuracy"])
print("Precision:", eval_metrics["eval_precision"])
print("Recall:", eval_metrics["eval_recall"])
print("F1-Score:", eval_metrics["eval_f1"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of LongformerForQuestionAnswering were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ArrowInvalid: Column 3 named start_positions expected length 4096 but got length 1

In [ ]:
 # a function to answer a question using the fine-tuned model
def answer_question(question, context_text):
    # Tokenize question and context
    inputs = tokenizer(question, context_text, max_length=4096, truncation=True, padding="max_length", return_tensors="pt")
    # Perform inference to generate an answer
    outputs = model(**inputs)
    # Decode the answer from the model's output
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end])
    return answer

# Provide a question and context for which we want to generate an answer
question = "What skills does the candidate have?"
context_text = "The candidate has experience in machine learning, natural language processing, and deep learning."

# Get the answer
answer = answer_question(question, context_text)
print("Answer:", answer)

Answer: 
